In [1]:
#TODO Costs and fees

In [2]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import matplotlib
from scipy.stats import pearsonr

In [3]:
targetExposure = 1_000_000.00
marginFactor = 0.05
totalSupply = 1_000_000

In [4]:
def getDollarExposure(spotEntry, noShares):
  return spotEntry * noShares * marginFactor

def getSharesForTargetExposure(underlyingPrice):
  return int(targetExposure / underlyingPrice)

In [5]:
def get_token_nav(o, c):
    targetShares = getSharesForTargetExposure(o)
    dollarExposure = getDollarExposure(float(o), targetShares)
    CFDnotionalTokenValue = (dollarExposure * targetShares) / totalSupply
    return CFDnotionalTokenValue

In [6]:
df = pd.read_csv("./data/USO.csv").set_index('Date').reset_index()

In [7]:
df['shares_traded'] = df.apply(lambda row: getSharesForTargetExposure(row['Open']), axis = 1)
df['token_val_eod'] = df.apply (lambda row: get_token_nav(row['Open'], row['Close']), axis=1)
df['token_daily_return'] = df.apply (lambda row: get_token_nav(row['Open'], None) - get_token_nav(row['Close'], None) , axis=1)

In [8]:
# dollar value of CFD portfolio
df['dollar_exp_o'] = df.apply(lambda row: getDollarExposure(row['Open'], row['shares_traded']), axis=1)
df['notional_exp_o'] = df.apply(lambda row: row['dollar_exp_o'] / marginFactor, axis = 1)

df['dollar_exp_c'] = df.apply(lambda row: getDollarExposure(row['Close'], row['shares_traded']), axis=1)
df['notional_exp_c'] = df.apply(lambda row: row['dollar_exp_c'] / marginFactor, axis = 1)

In [9]:
df

,Date,Open,High,Low,Close,Adj Close,Volume,shares_traded,token_val_eod,token_daily_return,dollar_exp_o,notional_exp_o,dollar_exp_c,notional_exp_c
0,2021-11-23,55.450001,56.270000,55.439999,56.200001,56.200001,5877400,18034,901.686761,12.066460,49999.265902,999985.318034,50675.540902,1.013511e+06
1,2021-11-24,55.810001,56.380001,55.779999,55.880001,55.880001,4109000,17917,895.803226,1.077726,49997.389396,999947.787917,50060.098896,1.001202e+06
2,2021-11-26,52.330002,52.380001,48.560001,49.630001,49.630001,18765600,19109,955.425166,-52.019686,49998.700411,999974.008218,47418.984455,9.483797e+05
3,2021-11-29,51.970001,52.169998,49.639999,50.169998,50.169998,13571400,19241,962.006505,-34.581935,49997.739462,999954.789241,48266.046576,9.653209e+05
4,2021-11-30,48.619999,49.110001,46.470001,47.910000,47.910000,21152500,20567,1028.316599,-15.259941,49998.375972,999967.519433,49268.248500,9.853650e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,2022-11-16,72.110001,72.470001,71.379997,72.040001,72.040001,3359600,13867,693.314905,-0.726248,49997.469193,999949.383867,49948.934693,9.989787e+05
248,2022-11-17,71.150002,71.440002,69.519997,70.139999,70.139999,5000100,14054,702.659333,-10.180662,49997.106405,999942.128108,49287.377297,9.857475e+05
249,2022-11-18,67.730003,69.120003,67.459999,69.029999,69.029999,8371700,14764,738.174727,13.897495,49998.288215,999965.764292,50957.945262,1.019159e+06
250,2022-11-21,66.669998,69.370003,65.370003,69.029999,69.029999,12862900,14999,749.937476,25.660244,49999.165000,999983.300002,51769.047750,1.035381e+06


# WTF

Ok, so basicall there have to exist two DFs: one should concentrate on the theoretical portfolio of daily cfds.

In [10]:
cdf_returns = np.log(df['cfd_portfolio_returns'] / np.linalg.norm(df['cfd_portfolio_returns']))
index_returns = np.log(df['Close']/np.linalg.norm(df['Close']))
plt.plot(cdf_returns, linestyle="-")
plt.plot(index_returns, linestyle = '-')
plt.show()

KeyError: 'cfd_portfolio_returns'

# CFD synthetic portfolio
ignoring margins

In [ ]:
df.loc[0, 'cfd_portfolio_returns'] = df.loc[0, 'cfd_pl']

In [ ]:
for i in range(1, len(df)):
    df.loc[i, 'cfd_portfolio_returns'] = df.loc[i-1, 'cfd_portfolio_returns'] + df.loc[i, 'cfd_pl']

In [ ]:
df.plot.line(y = 'cfd_portfolio_returns')